# test 1 biobert

In [150]:
from ast import literal_eval
from itertools import chain
from sklearn.metrics import precision_recall_fscore_support
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import StratifiedKFold
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm

In [34]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
# load data
import pandas as pd
df = pd.read_csv('../data/raw/mtsamples.csv')
df.transcription=df.transcription.astype(str)
#print(df.columns)
df_test = df.head(15)
df_test.head(2)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."


In [62]:
def convert(lst):
    '''This function converts the keywords to a list of keywords'''
    return ([i for i in lst.split()])
df_test['keywords']=df_test['keywords'].fillna("")
df_test['keywords_list']= df_test['keywords'].apply(lambda x: x.split())
#ACHTUNG HIER IST ES IMPORTANT IN EINEM NEXT STEP DIE KEYWORD LIST ZU CLEANEN
df_test.head(2)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...","[allergy, /, immunology,, allergic, rhinitis,,..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...","[bariatrics,, laparoscopic, gastric, bypass,, ..."


In [66]:
def location_indices(stringstext, check_list):
    '''this function finds the location of the keywords in the string'''
    
    res = dict()
    for ele in check_list :
        if ele in stringstext:
         
            # getting front index
            strt = stringstext.index(ele)
            
            # getting ending index
            res[ele] = [strt, strt + len(ele) - 1]
    return res.values()
        
df_test['col_3'] = df_test.apply(lambda x: location_indices(x.transcription, x.keywords_list), axis=1) 
df_test.head(2)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list,col_3
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...","[allergy, /, immunology,, allergic, rhinitis,,...","([808, 808], [491, 495])"
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...","[bariatrics,, laparoscopic, gastric, bypass,, ...","([1407, 1412], [1402, 1405], [1473, 1476])"


In [67]:
#new try Kfold
import numpy as np
frames = []
df_split = np.array_split(df_test, 5)
for split in range(0, 5):
    df_split[split]['kfold'] = split
    frames.append(df_split[split])
dfx = pd.concat(frames)
dfx

#find max len der texte
max_len = df_test['transcription'].map(lambda x: len(x)).max()
max_len

4422

In [73]:
#configuration

class config:
    MAX_LEN = 416
    TRAIN_BATCH_SIZE = 8
    VALID_BATCH_SIZE = 8
    EPOCHS = 5
    
    #tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    #model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

    
    
    BERT_PATH = 'emilyalsentzer/Bio_ClinicalBERT'
    #BERT_PATH = "../input/bert-base-uncased/" 
    #MODEL_PATH = "model.bin"
    MODEL_PATH = 'emilyalsentzer/Bio_ClinicalBERT'
    #tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    #TOKENIZER = tokenizers.BertWordPieceTokenizer(f"{BERT_PATH}/vocab.txt" ,lowercase = True)
    TOKENIZER = AutoTokenizer.from_pretrained( MODEL_PATH,lowercase = True)
    DROPOUT = 0.2
    MAX_GRAD_NORM = 1.0
    LEARNING_RATE = 1e-5

In [69]:
#Data Processing
first = df_test.loc[3]
example = {
    "feature_text": first.transcription,
    "location": first.col_3,
    "annotation": first.keywords_list
}
for key in example.keys():
    print(key)
    print(example[key])
    print("=" * 100)

feature_text
2-D M-MODE: , ,1.  Left atrial enlargement with left atrial diameter of 4.7 cm.,2.  Normal size right and left ventricle.,3.  Normal LV systolic function with left ventricular ejection fraction of 51%.,4.  Normal LV diastolic function.,5.  No pericardial effusion.,6.  Normal morphology of aortic valve, mitral valve, tricuspid valve, and pulmonary valve.,7.  PA systolic pressure is 36 mmHg.,DOPPLER: , ,1.  Mild mitral and tricuspid regurgitation.,2.  Trace aortic and pulmonary regurgitation.
location
dict_values([[290, 295], [297, 302], [24, 29], [216, 224], [176, 183], [304, 309], [243, 253], [339, 347], [136, 143], [318, 326], [299, 300]])
annotation
['cardiovascular', '/', 'pulmonary,', '2-d', 'm-mode,', 'doppler,', 'aortic', 'valve,', 'atrial', 'enlargement,', 'diastolic', 'function,', 'ejection', 'fraction,', 'mitral,', 'mitral', 'valve,', 'pericardial', 'effusion,', 'pulmonary', 'valve,', 'regurgitation,', 'systolic', 'function,', 'tricuspid,', 'tricuspid', 'valve,', 

In [70]:
#is this really needed? - if it is we have list of lists
def loc_list_to_ints(loc_list):
    '''this function makes the locations to integers'''
    to_return = []
    for loc_str in loc_list:
        loc_strs = loc_str.split(";")
        for loc in loc_strs:
            start, end = loc.split()
            to_return.append((int(start), int(end)))
    return to_return

example_loc_ints = loc_list_to_ints(example["location"])

AttributeError: 'list' object has no attribute 'split'

In [122]:
def process_data_tokenize(feature_text, annotation, location, tokenizer, max_len):    ##X , Y, selected_text  
    feature_text = feature_text[:40]
    location_list = location#loc_list_to_ints(location)        
    char_targets = [0] * len(feature_text) #creating empty list(all zeros) of character;it will be made 1 if annotation in text   
    
    for loc,anno in zip(location_list,annotation): 
        len_st = loc[1] - loc[0]

        idx0 = None
        idx1 = None
        for ind in (i for i, e in enumerate(feature_text) if (e == anno[0] and i == loc[0])):
            if feature_text[ind: ind+len_st] == anno:

                idx0 = ind
                idx1 = ind + len_st - 1
                if idx0 != None and idx1 != None:
                    for ct in range(idx0, idx1 + 1):
                        char_targets[ct] = 1  #replacing zeros with 1 if that part of the text is selected text
        
                break
    tokenized_input = tokenizer.encode_plus(feature_text, #max_len=40, 
                                            add_special_tokens=True, 
                                            return_attention_mask=True, 
                                            return_offsets_mapping=True,
                                            return_tensors = 'pt')
       # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    
    #for key, value in tokenized_input.items():
        #print( '{} : {}'.format( key, value ) )
    input_ids = tokenized_input.get('input_ids')
    mask = tokenized_input.get('attention_mask')
    token_type_ids = tokenized_input.get('token_type_ids')
    offsets = tokenized_input.get('offset_mapping')
        
    
    #input_ids = tokenized_input.ids
    #mask = tokenized_input.attention_mask
    #token_type_ids = tokenized_input.type_ids
    #offsets = tokenized_input.offsets
    #print(offsets)
    #print('++++')
    target_idx = []
    for value in offsets:
        for j, (offset1, offset2) in enumerate(value):
            #print(j, offset1, offset2)
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)
            
    #padding not working
    padding_length = 0# max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        offsets = offsets + ([(0, 0)] * padding_length)
       
    #creating label
    ignore_idxes = np.where(np.array(token_type_ids) != 1)[0]

    label = np.zeros(len(offsets))
    label[ignore_idxes] = -1
    label[target_idx] = 1

    
    return {
    'ids': input_ids,
    'mask': mask,
    'token_type_ids': token_type_ids,
    'labels': label,
    'offsets': offsets
}

In [154]:
example.keys()

dict_keys(['feature_text', 'location', 'annotation'])

In [123]:
output = process_data_tokenize(example["feature_text"],example["annotation"],example["location"],config.TOKENIZER,config.MAX_LEN)

for key in output.keys():
    print(key)
    print(output[key])
    print("=" * 100)

ids
tensor([[  101,   123,   118,   173,   182,   118,  5418,   131,   117,   117,
           122,   119,  1286,  1120, 13119,  4035,  5815,  2176,  3263,   102]])
mask
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
token_type_ids
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
labels
[-1.]
offsets
tensor([[[ 0,  0],
         [ 0,  1],
         [ 1,  2],
         [ 2,  3],
         [ 4,  5],
         [ 5,  6],
         [ 6, 10],
         [10, 11],
         [12, 13],
         [14, 15],
         [15, 16],
         [16, 17],
         [19, 23],
         [24, 26],
         [26, 30],
         [31, 33],
         [33, 36],
         [36, 38],
         [38, 40],
         [ 0,  0]]])


In [151]:
#Data loader
class NBMEDataset:
    def __init__(self,  feature_text, annotation, location):   #text(X) #label(Y), #selected_text #start:end
        #self.pn_history = pn_history
        self.feature_text = feature_text
        self.annotation = annotation
        self.location = location
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
        
    def __len__(self):
        return len(self.feature_text)
        
    def __getitem__(self, item):
        data = process_data_tokenize(
            #self.pn_history[item],
            self.feature_text[item],
            self.annotation[item],
            self.location[item],
            self.tokenizer,
            self.max_len
        )
        print('datafunction')
        print(torch.tensor(data["ids"]))
        return {
            'ids': torch.tensor(data["ids"]), #input_ids
            'mask': torch.tensor(data["mask"], dtype=torch.long), #attention_mask
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long), #segment_ids
            'labels': torch.tensor(data["labels"], dtype=torch.long), 
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }
        

In [159]:
output = NBMEDataset(example["feature_text"],example["annotation"],example["location"])
print(output)#.__getitem__()

train_data_loader = torch.utils.data.DataLoader(
        output,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4
    )

train_data_loader

In [132]:
import transformers
class NBMEModel(transformers.BertPreTrainedModel):    #torch.nn.Module
    def __init__(self,conf):
        super(NBMEModel,self).__init__(conf)
        self.bert = transformers.BertModel.from_pretrained(config.BERT_PATH, config=conf)
        self.dropout = torch.nn.Dropout(config.DROPOUT)
        self.classifier = torch.nn.Linear(768, 1)
        torch.nn.init.normal_(self.classifier.weight, std=0.02) 
        
    def forward(self, ids, mask, token_type_ids):
        sequence_out = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)[0] #last_hidden_state
        batch_size,max_len,feat_dim = sequence_out.shape
        sequence_output = self.dropout(sequence_out)
        logits = self.classifier(sequence_output)
        logits = logits.squeeze(-1) 
        return logits

In [133]:
#utility function
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
#loss function
def loss_fn(logits, labels):
    loss_fct = torch.nn.BCEWithLogitsLoss(reduction = "none")
    loss = loss_fct(logits,labels)
    return loss

In [148]:
#training function
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def train_fn(dataloader, model, optimizer, scheduler=None):
    model.train()
    losses = AverageMeter() 
    print('dataloader', len(dataloader))
    # Computes and stores the average and current value
    tk = tqdm(dataloader, total=len(dataloader)) 
    #tqdm is a Python library for adding progress bar. 
    print('in function')
    print(tk)
    for batch, data in enumerate(tk):
        print(data)
        #check hier
        #ids = data['ids']
        token_type_ids = data["token_type_ids"]
        #mask = data["mask"]
        #labels = data["labels"]
        #offsets = data["offsets"]
        
        #adding the below data to device ;device enables you to specify the device type responsible to load a tensor into memory.
        ids = ids.to(DEVICE, dtype=torch.long)
        token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
        mask = mask.to(DEVICE, dtype=torch.long)
        labels = labels.to(DEVICE, dtype=torch.float64)

        model.zero_grad()
        logits = model(ids=ids, mask=mask, token_type_ids=token_type_ids) #last_hidden_state

        loss = loss_fn(logits, labels)
        loss = torch.masked_select(loss, labels > -1.0).mean()
        losses.update(loss.item(),ids.size(0))
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), config.MAX_GRAD_NORM)
        optimizer.step()
        scheduler.step() ## Update learning rate schedule
        
        #output = torch.argmax(torch.softmax(logits, dim=2),dim=2).cpu().detach().numpy()
        tk.set_postfix(loss=losses.avg)
        
    return losses.avg

In [149]:
#evalutation method missing
#training
import time
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
def run(fold):
    
    train_loss_data, valid_loss_data = [], []
    best_loss = np.inf
    since = time.time()
   
    df_train = dfx[dfx.kfold != fold].reset_index(drop=True) 
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)
    
    train_dataset = NBMEDataset(
        #pn_history=df_train.pn_history.values,
        feature_text=df_train.transcription.values,
        annotation=df_train.keywords_list.values,
        location=df_train.col_3.values
        
    )
    
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4
    )

    valid_dataset = NBMEDataset(
        #pn_history=df_valid.pn_history.values,
        feature_text=df_valid.transcription.values,
        annotation=df_valid.keywords_list.values,
        location=df_valid.col_3.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2
    )

    model_config = transformers.BertConfig.from_pretrained(config.BERT_PATH)
    model_config.output_hidden_states = True
    model = NBMEModel(conf=model_config)
    model.to(DEVICE)
    
    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=config.LEARNING_RATE)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

    best_loss = np.inf
    
    for i in range(config.EPOCHS):
        print("Epoch: {}/{}".format(i + 1, config.EPOCHS))
    
        # train model
        train_loss = train_fn(train_data_loader
                              , model, optimizer, scheduler=scheduler)
        train_loss_data.append(train_loss)
        print(f"Train loss: {train_loss}")

        # evaluate model - not now
      #  valid_loss = eval_fn(valid_data_loader, model)
      #  valid_loss_data.append(valid_loss)
      #  print(f"Valid loss: {valid_loss}")


       # if valid_loss < best_loss:
        #    best_loss = valid_loss
        torch.save(model.state_dict(), "model_fold1.bin")


        #time_elapsed = time.time() - since
        #print('Training completed in {:.0f}m {:.0f}s'.format(
        #    time_elapsed // 60, time_elapsed % 60))
    
    
run(fold=0)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 1/5
dataloader 2


/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

in function
  0%|          | 0/2 [00:00<?, ?it/s]


TypeError: cannot pickle 'dict_values' object

In [ ]:
######gut #######

In [39]:
def create_train_df():
    #feats = pd.read_csv(f"{CFG.root}/features.csv")
    #notes = pd.read_csv(f"{CFG.root}/patient_notes.csv")
    train = pd.read_csv(f"{CFG.root}")
    merged = train.head(20)

    #train["annotation_list"] = [literal_eval(x) for x in train["annotation"]]
    #train["location_list"] = [literal_eval(x) for x in train["location"]]
    #merged = train.merge(notes, how = "left")
    #merged = merged.merge(feats, how = "left")
    #merged = merged.loc[merged["annotation"] != "[]"].copy().reset_index(drop = True) # comment out if you train all samples
    
    def process_feature_text(text):
        return text.replace("-OR-", ";-").replace("-", " ")
    merged["transcription"] = [process_feature_text(x) for x in merged["transcription"]]
    
    merged["transcription"] = merged["transcription"].apply(lambda x: x.lower())
    #merged["pn_history"] = merged["pn_history"].apply(lambda x: x.lower())
    
    merged['location_prediction'] = -1
    merged['token_proba'] = -1
    merged['token_offsets'] = -1

    if CFG.debug:
        merged = merged.sample(frac = 0.5).reset_index(drop = True)

    skf = StratifiedKFold(CFG.n_fold)
    #merged["stratify_on"] = merged.index.astype(str) #+ merged["feature_num"].astype(str)
    #merged["fold"] = -1
    #for fold, (_, valid_idx) in enumerate(skf.split(merged["id"], y = merged["stratify_on"])):
    #    merged.loc[valid_idx, "fold"] = fold
    
    #print(merged.shape)
    return merged

df = create_train_df()
df

/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_2439/195158292.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["transcription"] = [process_feature_text(x) for x in merged["transcription"]]
/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_2439/195158292.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["transcription"] = merged["transcription"].apply(lambda x: x.lower())
/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_2439/195158292.py:20: SettingWithCopy

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,location_prediction,token_proba,token_offsets
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"subjective:, this 23 year old white female pr...","allergy / immunology, allergic rhinitis, aller...",-1,-1,-1
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"past medical history:, he has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...",-1,-1,-1
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"history of present illness: , i have seen abc ...","bariatrics, laparoscopic gastric bypass, heart...",-1,-1,-1
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2 d m mode: , ,1. left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple...",-1,-1,-1
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. the left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo...",-1,-1,-1
5,5,Morbid obesity. Laparoscopic antecolic anteg...,Bariatrics,Laparoscopic Gastric Bypass,"preoperative diagnosis: , morbid obesity.,post...","bariatrics, gastric bypass, eea anastomosis, r...",-1,-1,-1
6,6,"Liposuction of the supraumbilical abdomen, re...",Bariatrics,Liposuction,"preoperative diagnoses:,1. deformity, right b...","bariatrics, breast reconstruction, excess, lma...",-1,-1,-1
7,7,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 3,"2 d echocardiogram,multiple views of the heart...","cardiovascular / pulmonary, 2-d echocardiogram...",-1,-1,-1
8,8,Suction-assisted lipectomy - lipodystrophy of...,Bariatrics,Lipectomy - Abdomen/Thighs,"preoperative diagnosis: , lipodystrophy of the...","bariatrics, lipodystrophy, abd pads, suction-a...",-1,-1,-1
9,9,Echocardiogram and Doppler,Cardiovascular / Pulmonary,2-D Echocardiogram - 4,"description:,1. normal cardiac chambers size....","cardiovascular / pulmonary, ejection fraction,...",-1,-1,-1


In [40]:
first = df.loc[0]
example = {
    "transcription": first.transcription,
    "pn_history": first.medical_specialty
   # "location_list": first.location_list,
    #"annotation_list": first.annotation_list
}
for key in example.keys():
    print(key)
    print(example[key])
    print("=" * 100)

transcription
subjective:,  this 23 year old white female presents with complaint of allergies.  she used to have allergies when she lived in seattle but she thinks they are worse here.  in the past, she has tried claritin, and zyrtec.  both worked for short time but then seemed to lose effectiveness.  she has used allegra also.  she used that last summer and she began using it again two weeks ago.  it does not appear to be working very well.  she has used over the counter sprays but no prescription nasal sprays.  she does have asthma but doest not require daily medication for this and does not think it is flaring up.,medications: , her only medication currently is ortho tri cyclen and the allegra.,allergies: , she has no known medicine allergies.,objective:,vitals:  weight was 130 pounds and blood pressure 124/78.,heent:  her throat was mildly erythematous without exudate.  nasal mucosa was erythematous and swollen.  only clear drainage was seen.  tms were clear.,neck:  supple without

In [43]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [49]:
def tokenize_and_add_labels(tokenizer, example):
    tokenized_inputs = tokenizer(
        example["transcription"],
        max_length = CFG.max_length,
        stride = CFG.doc_stride,
        padding = "max_length",
        truncation = "only_second",
        return_offsets_mapping = True
    )
    labels = [0.0] * len(tokenized_inputs["input_ids"])
    tokenized_inputs["sequence_ids"] = tokenized_inputs.sequence_ids()
    tokenized_inputs["labels"] = labels
    
    # for training
    for idx, (seq_id, offsets) in enumerate(zip(tokenized_inputs["sequence_ids"], tokenized_inputs["offset_mapping"])):
        if seq_id is None or seq_id == 0:
            labels[idx] = -100
            continue
        exit = False
        token_start, token_end = offsets
        for feature_start, feature_end in tokenized_inputs.index: #tokenized_inputs["location_int"]:
            if exit:
                break
            if token_start >= feature_start and token_end <= feature_end:
                labels[idx] = 1.0
                exit = True
    tokenized_inputs["labels"] = labels
    
    
    return tokenized_inputs

tokenized_inputs = tokenize_and_add_labels(tokenizer, example)
for key in tokenized_inputs.keys():
    print(key)
    print(tokenized_inputs[key])
    print("=" * 100)

input_ids
[101, 23481, 131, 117, 1142, 1695, 1214, 1385, 1653, 2130, 8218, 1114, 12522, 1104, 1155, 1200, 19310, 119, 1131, 1215, 1106, 1138, 1155, 1200, 19310, 1165, 1131, 2077, 1107, 1946, 5034, 1133, 1131, 6191, 1152, 1132, 4146, 1303, 119, 1107, 1103, 1763, 117, 1131, 1144, 1793, 172, 5815, 17030, 1179, 117, 1105, 195, 12577, 19000, 119, 1241, 1589, 1111, 1603, 1159, 1133, 1173, 1882, 1106, 3857, 12949, 119, 1131, 1144, 1215, 1155, 12606, 1611, 1145, 119, 1131, 1215, 1115, 1314, 2247, 1105, 1131, 1310, 1606, 1122, 1254, 1160, 2277, 2403, 119, 1122, 1674, 1136, 2845, 1106, 1129, 1684, 1304, 1218, 119, 1131, 1144, 1215, 1166, 1103, 4073, 13477, 1116, 1133, 1185, 24259, 21447, 13477, 1116, 119, 1131, 1674, 1138, 1112, 1582, 1918, 1133, 1674, 1204, 1136, 4752, 3828, 15683, 1111, 1142, 1105, 1674, 1136, 1341, 1122, 1110, 22593, 10832, 1146, 119, 117, 23897, 131, 117, 1123, 1178, 15683, 1971, 1110, 1137, 1582, 1186, 189, 2047, 5120, 1179, 1105, 1103, 1155, 12606, 1611, 119, 117, 1155, 12

In [50]:
#Dataset
import numpy as np
class NBMEData(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        example = self.data.loc[idx]
        tokenized = tokenize_and_add_labels(self.tokenizer, example)

        input_ids = np.array(tokenized["input_ids"]) # for input BERT
        attention_mask = np.array(tokenized["attention_mask"]) # for input BERT
        labels = np.array(tokenized["labels"]) # for calculate loss and cv score

        offset_mapping = np.array(tokenized["offset_mapping"]) # for calculate cv score
        sequence_ids = np.array(tokenized["sequence_ids"]).astype("float16") # for calculate cv score
        
        return input_ids, attention_mask, labels, offset_mapping, sequence_ids

In [55]:
#model
class NBMEModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

        self.backbone = AutoModel.from_pretrained(model) # BERT model
        self.dropout = torch.nn.Dropout(p = 0.2)
        self.classifier = torch.nn.Linear(768, 1) # BERT has last_hidden_state(size: sequqence_length, 768)
    
    def forward(self, input_ids, attention_mask):
        last_hidden_state = self.backbone(input_ids = input_ids, attention_mask = attention_mask)[0] # idx 0 is last_hidden_state; backbone().last_hidden_state is also good
        logits = self.classifier(self.dropout(last_hidden_state)).squeeze(-1)
        return logits

In [52]:
#Training
def train_loop(fold):
    model = NBMEModel().to(CFG.device)
    #criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), CFG.lr)

    train = df.loc[df["fold"] != fold].reset_index(drop = True)
    valid = df.loc[df["fold"] == fold].reset_index(drop = True)
    train_ds = NBMEData(train, tokenizer)
    valid_ds = NBMEData(valid, tokenizer)
    train_dl = torch.utils.data.DataLoader(train_ds, batch_size = CFG.batch_size, pin_memory = True, shuffle = True, drop_last = True)
    valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size = CFG.batch_size * 2, pin_memory = True, shuffle = False, drop_last = False)
    
    return train_dl, valid_dl, model, optimizer

In [53]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def get_location_predictions(preds, offset_mapping, sequence_ids, test = False):
    all_predictions = []
    for pred, offsets, seq_ids in zip(preds, offset_mapping, sequence_ids):
        pred = sigmoid(pred)
        start_idx = None
        current_preds = []
        for p, o, s_id in zip(pred, offsets, seq_ids):
            if s_id is None or s_id == 0:
                continue
            if p > 0.75:
                if start_idx is None:
                    start_idx = o[0]
                end_idx = o[1]
            elif start_idx is not None:
                if test:
                    current_preds.append(f"{start_idx} {end_idx}")
                else:
                    current_preds.append((start_idx, end_idx))
                start_idx = None
        if test:
            all_predictions.append("; ".join(current_preds))
        else:
            all_predictions.append(current_preds)
    return all_predictions

def calculate_char_CV(predictions, offset_mapping, sequence_ids, labels):
    all_labels = []
    all_preds = []
    for preds, offsets, seq_ids, labels in zip(predictions, offset_mapping, sequence_ids, labels):
        num_chars = max(list(chain(*offsets)))
        char_labels = np.zeros((num_chars))
        for o, s_id, label in zip(offsets, seq_ids, labels):
            if s_id is None or s_id == 0:
                continue
            if int(label) == 1:
                char_labels[o[0]:o[1]] = 1
        char_preds = np.zeros((num_chars))
        for start_idx, end_idx in preds:
            char_preds[start_idx:end_idx] = 1
        all_labels.extend(char_labels)
        all_preds.extend(char_preds)
    results = precision_recall_fscore_support(all_labels, all_preds, average = "binary")
    return {
        "precision": results[0],
        "recall": results[1],
        "f1": results[2]
    }

In [56]:
def model_loop():    
    history = {}
    for fold in range(CFG.n_fold):
        print(f"========== fold: {fold} training ==========")
        train_dl, valid_dl, model, optimizer = train_loop(fold)
        history[fold] = {"train": [], "valid": []}
        best_loss = np.inf
        
        for epoch in range(CFG.epochs):
            print(f"========== EPOCH: {epoch} training ==========")
            #training
            model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

            model.train()
            train_loss = AverageMeter()
            pbar = tqdm(train_dl)
            for batch in pbar:
                optimizer.zero_grad()
                input_ids = batch[0].to(CFG.device)
                attention_mask = batch[1].to(CFG.device)
                labels = batch[2].to(CFG.device)
                offset_mapping = batch[3]
                sequence_ids = batch[4]
                logits = model(input_ids, attention_mask)
                loss_fct = torch.nn.BCEWithLogitsLoss(reduction = "none")
                loss = loss_fct(logits, labels)
                loss = torch.masked_select(loss, labels > -1).mean() # we should calculate at "pn_history"; labels at "feature_text" are -100 < -1
                loss.backward()
                optimizer.step()
                train_loss.update(val = loss.item(), n = len(input_ids))
                pbar.set_postfix(Loss = train_loss.avg)
            print(epoch, train_loss.avg)
            history[fold]["train"].append(train_loss.avg)

            #evaluation
            model.eval()
            valid_loss = AverageMeter()
            preds = []
            offsets = []
            seq_ids = []
            lbls = []
            with torch.no_grad():
                for batch in tqdm(valid_dl):
                    input_ids = batch[0].to(CFG.device)
                    attention_mask = batch[1].to(CFG.device)
                    labels = batch[2].to(CFG.device)
                    offset_mapping = batch[3]
                    sequence_ids = batch[4]
                    logits = model(input_ids, attention_mask)
                    loss_fct = torch.nn.BCEWithLogitsLoss(reduction = "none")
                    loss = loss_fct(logits, labels)
                    loss = torch.masked_select(loss, labels > -1).mean()
                    valid_loss.update(val = loss.item(), n = len(input_ids))
                    pbar.set_postfix(Loss = valid_loss.avg)
                    preds.append(logits.cpu().numpy())
                    offsets.append(offset_mapping.numpy())
                    seq_ids.append(sequence_ids.numpy())
                    lbls.append(labels.cpu().numpy())
            print(epoch, valid_loss.avg)
            history[fold]["valid"].append(valid_loss.avg)          
            
            # save model
            if valid_loss.avg < best_loss:
                best_loss = valid_loss.avg
                torch.save(model.state_dict(), f"nbme_{fold}.pth")
                preds = np.concatenate(preds, axis = 0)
                # print(preds.shape)
                # print([preds[i][0] for i in range(preds.shape[0])])
                # print(preds)
                offsets = np.concatenate(offsets, axis = 0)
                seq_ids = np.concatenate(seq_ids, axis = 0)
                lbls = np.concatenate(lbls, axis = 0)
                location_preds= get_location_predictions(preds, offsets, seq_ids)
                # print(offsets.shape)
                # df.loc[df['fold'] == fold, 'location_prediction'] = written_predictions
                index = df[df['fold'] == fold].index
                df.loc[index,'location_prediction'] = pd.Series(location_preds, index=index)
                df.loc[index,'token_proba'] = pd.Series([list(preds[i]) for i in range(preds.shape[0])], index=index)
                df.loc[index,'token_offsets'] = pd.Series([list(offsets[i]) for i in range(offsets.shape[0])], index=index)
                score = calculate_char_CV(location_preds, offsets, seq_ids, lbls)
                # if epoch == 1:
                #     return location_preds
                print(score)
    print(history)
    return()
model_loop()

========== fold: 0 training ==========


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OSError: Can't load the configuration of 'BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout)

In [ ]:
https://www.kaggle.com/code/daisybbb/pytorch-pubmedbert-infer
https://www.kaggle.com/code/tanyadayanand/nbme-bert-base-uncased-using-pytorch

In [41]:
#not working
tokenizer = AutoTokenizer.from_pretrained(CFG.model)

# Questions: 
# 1. why not use doc_stride  -> treated
# 2. why using duoble instead of int -> treated
def tokenize_and_add_labels(tokenizer, example):
    tokenized_inputs = tokenizer(
        example["transcription"],
        max_length = CFG.max_length,
        stride = CFG.doc_stride,
        padding = "max_length",
        truncation = "only_second",
        return_offsets_mapping = True
    )
    labels = [0.0] * len(tokenized_inputs["input_ids"])
    #tokenized_inputs["location_int"] = loc_list_to_ints(example["location_list"])
    tokenized_inputs["sequence_ids"] = tokenized_inputs.sequence_ids()

    for idx, (seq_id, offsets) in enumerate(zip(tokenized_inputs["sequence_ids"], tokenized_inputs["offset_mapping"])):
        if seq_id is None or seq_id == 0:
            labels[idx] = -100
            continue
        exit = False
        token_start, token_end = offsets
        for feature_start, feature_end in tokenized_inputs["location_int"]:
            if exit:
                break
            if token_start >= feature_start and token_end <= feature_end:
                labels[idx] = 1.0
                exit = True
    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs


HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_feat

In [2]:
import pandas as pd
from transformers import BertTokenizerFast


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#from transformers import BertModel
#bert = BertModel.from_pretrained('bert-base-uncased')
from transformers import BertTokenizer,BertForTokenClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.add_tokens(['B_geo','I_geo','B_per','I_per','B_org','I_org'])

In [15]:
#load data
# load data
df = pd.read_csv('../data/raw/mtsamples.csv')
df.transcription=df.transcription.astype(str)
#print(df.columns)
df.head(15)
df_test = df.head(15)

In [4]:
#difference between berttikenizerfast and autotokenizer
example = " I have an issue my back hurts my nose is bleeding i am hungry."
tokenizer = BertTokenizerFast.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
text_tokenized = tokenizer(example, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
#tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
text_tokenized

{'input_ids': tensor([[  101,   178,  1138,  1126,  2486,  1139,  1171, 15483,  1139,  3678,
          1110,  9793,   178,  1821,  7555,   119,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [5]:
#decoding

print(tokenizer.decode(text_tokenized.input_ids[0]))

[CLS] i have an issue my back hurts my nose is bleeding i am hungry. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [6]:
print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))

['[CLS]', 'i', 'have', 'an', 'issue', 'my', 'back', 'hurts', 'my', 'nose', 'is', 'bleeding', 'i', 'am', 'hungry', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [7]:
word_ids = text_tokenized.word_ids()
print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))
print(word_ids)

['[CLS]', 'i', 'have', 'an', 'issue', 'my', 'back', 'hurts', 'my', 'nose', 'is', 'bleeding', 'i', 'am', 'hungry', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [ ]:
#no use yet we need label output also class for batch processing
def align_label_example(tokenized_input, labels):

        word_ids = tokenized_input.word_ids()

        previous_word_idx = None
        label_ids = []
   
        for word_idx in word_ids:

            if word_idx is None:
                label_ids.append(-100)
                
            elif word_idx != previous_word_idx:
                try:
                  label_ids.append(labels_to_ids[labels[word_idx]])
                except:
                  label_ids.append(-100)
        
            else:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
      

        return label_ids

In [8]:
#train test split
# evaluaiton - https://medium.com/nwamaka-imasogie/clinicalbert-using-deep-learning-transformer-model-to-predict-hospital-readmission-c82ff0e4bb03
import numpy as np

df = df[0:1000]
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

In [11]:
from transformers import BertForTokenClassification
import torch
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=14)#len(unique_labels))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask)#, labels=label, return_dict=True)

        return output


model_bert = BertModel()
    
    
   # tokenizer = BertTokenizerFast.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
#text_tokenized = tokenizer(example, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
#tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

In [ ]:
import torch

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):

        lb = [i.split() for i in df['labels'].values.tolist()]
        txt = df['text'].values.tolist()
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

In [13]:
model_bert(df_train)

TypeError: BertModel.forward() missing 2 required positional arguments: 'mask' and 'label'

In [12]:
#https://towardsdatascience.com/named-entity-recognition-with-bert-in-pytorch-a454405e0b6a
import torch
def train_loop(model, df_train, df_val):

    train_dataset = df_train#DataSequence(df_train)
    val_dataset = df_val#DataSequence(df_val)

    #train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    #val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

LEARNING_RATE = 5e-3
EPOCHS = 5
BATCH_SIZE = 2

#model = BertModel()
train_loop(model_bert, df_train, df_val)

NameError: name 'DataSequence' is not defined

In [4]:
https://medium.com/analytics-vidhya/named-entity-recognition-for-turkish-with-bert-f8ec04a31b0
https://www.kaggle.com/code/mdfahimreshm/bert-in-depth-understanding


Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3/envs/nlp_masterthesis

  added / updated specs:
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2022.9.24          |  py310hecd8cb5_0         155 KB
    ffmpeg-4.3                 |       h0a44026_0        10.1 MB  pytorch
    gettext-0.21.0             |       h7535e17_0         2.6 MB
    gmp-6.2.1                  |       he9d5cce_3         525 KB
    gnutls-3.6.15              |       hed9c0bf_0         974 KB
    lame-3.100                 |       h1de35cc_0         316 KB
    libidn2-2.3.2              |       h9ed2024_0          85 KB
    libtasn1-4.16.0            |       h9ed2024_0          53 KB
    libunistring-0.9.10        |       h9ed2024_0         519 KB
    nettle-3.7.3               |       h230ac6f_1         380 KB
    openh264

In [ ]:
# github bio bert - https://github.com/dmis-lab/biobert/blob/master/modeling.py
https://www.kaggle.com/code/daisybbb/pytorch-pubmedbert
https://www.kaggle.com/code/tanyadayanand/nbme-bert-base-uncased-using-pytorch
https://www.kaggle.com/code/mdmustafijurrahman/bert-named-entity-recognition-ner-data

In [ ]:
# Import package
from wordcloud import WordCloud, STOPWORDS

# Generate word cloud
wordcloud = WordCloud(width=3000,
                      height=2000,
                      random_state=1, 
                      background_color='white',
                      colormap='Pastel1',
                      collocations=False,
                      stopwords=['-', 'or', 'of', 'with', 'ago'])\
                      .generate(body)
# Plot
plt.rcParams["figure.figsize"] = (15, 10)
plt.imshow(wordcloud)
plt.title('Wordcloud of features', fontsize=24)
plt.axis('off')
plt.savefig('./wordcloud.png')

# test 2 keyword berd

In [26]:
https://medium.com/geekculture/3-techniques-to-perform-named-entity-recognition-ner-on-text-data-ec1e91e3a8aa
from sklearn.feature_extraction.text import CountVectorizer

doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """


n_gram_range = (1, 3)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names()
candidates

/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['algorithm',
 'algorithm analyzes',
 'algorithm analyzes training',
 'algorithm correctly',
 'algorithm correctly determine',
 'algorithm generalize',
 'algorithm generalize training',
 'allow',
 'allow algorithm',
 'allow algorithm correctly',
 'analyzes',
 'analyzes training',
 'analyzes training data',
 'based',
 'based example',
 'based example input',
 'bias',
 'called',
 'called supervisory',
 'called supervisory signal',
 'class',
 'class labels',
 'class labels unseen',
 'consisting',
 'consisting input',
 'consisting input object',
 'consisting set',
 'consisting set training',
 'correctly',
 'correctly determine',
 'correctly determine class',
 'data',
 'data consisting',
 'data consisting set',
 'data produces',
 'data produces inferred',
 'data unseen',
 'data unseen situations',
 'desired',
 'desired output',
 'desired output value',
 'determine',
 'determine class',
 'determine class labels',
 'example',
 'example input',
 'example input output',
 'example pair',
 'examp

In [31]:
#Next, we convert both the document as well as the candidate keywords/keyphrases to numerical data.
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)
candidate_embeddings


array([[-1.2811013 , -0.19414134,  0.18961014, ..., -0.6902669 ,
        -0.08735337, -0.05631514],
       [-0.98599565, -0.3114873 ,  0.11392305, ..., -0.5420821 ,
         0.03869215,  0.04444062],
       [-0.7794865 , -0.3172609 ,  0.30012545, ..., -0.44612113,
         0.09859169,  0.41761875],
       ...,
       [-0.35578147,  0.01782149,  0.374094  , ..., -0.1977843 ,
         0.0460116 , -0.29936978],
       [-0.7603124 , -0.5274268 ,  0.29681763, ..., -0.3508133 ,
         0.16764224, -0.566111  ],
       [-0.6121529 , -0.44180587, -0.27497974, ..., -0.31791598,
         0.22374944, -0.78795713]], dtype=float32)

We are Distilbert as it has shown great performance in similarity tasks, which is what we are aiming for with keyword/keyphrase extraction!

Since transformer models have a token limit, you might run into some errors when inputting large documents. In that case, you could consider splitting up your document into paragraphs and mean pooling (taking the average of) the resulting vectors.

In the final step, we want to find the candidates that are most similar to the document. We assume that the most similar candidates to the document are good keywords/keyphrases for representing the document.

To calculate the similarity between candidates and the document, we will be using the cosine similarity between vectors as it performs quite well in high-dimensionality:

In [32]:

from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

['algorithm analyzes training',
 'learning algorithm generalize',
 'learning machine learning',
 'learning algorithm analyzes',
 'algorithm generalize training']

There is a reason why similar results are returned… they best represent the document! If we were to diversify the keywords/keyphrases then they are less likely to represent the document well as a collective.

Thus, the diversification of our results requires a delicate balance between the accuracy of keywords/keyphrases and the diversity between them.

There are two algorithms that we will be using to diversify our results:

Max Sum Similarity
Maximal Marginal Relevance
Max Sum Similarity
The maximum sum distance between pairs of data is defined as the pairs of data for which the distance between them is maximized. In our case, we want to maximize the candidate similarity to the document whilst minimizing the similarity between candidates.

To do this, we select the top 20 keywords/keyphrases, and from those 20, select the 5 that are the least similar to each other:
https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea

In [21]:
import numpy as np
import itertools

def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [23]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['maps', 'input', 'class', 'training', 'algorithm']

Maximal Marginal Relevance
The final method for diversifying our results is Maximal Marginal Relevance (MMR). MMR tries to minimize redundancy and maximize the diversity of results in text summarization tasks. Fortunately, a keyword extraction algorithm called EmbedRank has implemented a version of MMR that allows us to use it for diversifying our keywords/keyphrases.

We start by selecting the keyword/keyphrase that is the most similar to the document. Then, we iteratively select new candidates that are both similar to the document and not similar to the already selected keywords/keyphrases:

In [24]:

import numpy as np

def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [25]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['learning', 'algorithm', 'training', 'class', 'mapping']

In [ ]:
https://github.com/MaartenGr/KeyBERT/blob/master/keybert/_model.py
https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea

In [ ]:
https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea

from transformers import DistilBertModel
import torch
import torch.nn as nn
class DistilBERT_Model(nn.Module):
 def __init__(self, classes):
   super(DistilBERT_Model, self).__init__()
   self.distilbert = DistilBertModel.from_pretrained('distilbert
                                                     base-uncased')
   self.out = nn.Linear(self.distilbert.config.hidden_size, classes)
   self.sigmoid = nn.Sigmoid()
 def forward(self, input, attention_mask):
   _, output = self.distilbert(input, attention_mask 
                                      = attention_mask)
   out = self.sigmoid(self.out(output))
   return out